<a href="https://colab.research.google.com/github/JozefSL/pyNotes/blob/main/NorthDakota/hzWellsLateralsSTK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dbfread

In [2]:
import pandas as pd
import plotly.express as px
from dbfread import DBF
import ipywidgets as widgets
import csv
from math import radians, sin, cos, sqrt, atan2

In [3]:
# run command to download data.  In case it is not able to download, open the link and upload the data to this notebook from left hand side menu.
!wget https://gis.dmr.nd.gov/downloads/oilgas/shapefile/OGD_Horizontals.zip

--2023-09-07 13:57:55--  https://gis.dmr.nd.gov/downloads/oilgas/shapefile/OGD_Horizontals.zip
Resolving gis.dmr.nd.gov (gis.dmr.nd.gov)... 165.234.156.197
Connecting to gis.dmr.nd.gov (gis.dmr.nd.gov)|165.234.156.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 314215333 (300M) [application/x-zip-compressed]
Saving to: ‘OGD_Horizontals.zip’

OGD_Horizontals.zip 100%[===================>] 299.66M  57.8MB/s    in 5.4s    

2023-09-07 13:58:01 (55.6 MB/s) - ‘OGD_Horizontals.zip’ saved [314215333/314215333]



In [4]:
!unzip OGD_Horizontals.zip

Archive:  OGD_Horizontals.zip
  inflating: OGD_Horizontals.cpg     
  inflating: OGD_Horizontals.dbf     
  inflating: OGD_Horizontals.prj     
  inflating: OGD_Horizontals.sbn     
  inflating: OGD_Horizontals.sbx     
  inflating: OGD_Horizontals.shp     
  inflating: OGD_Horizontals.shp.xml  
  inflating: OGD_Horizontals.shx     
  inflating: OGD_Horizontals_Line.cpg  
  inflating: OGD_Horizontals_Line.dbf  
  inflating: OGD_Horizontals_Line.prj  
  inflating: OGD_Horizontals_Line.sbn  
  inflating: OGD_Horizontals_Line.sbx  
  inflating: OGD_Horizontals_Line.shp  
  inflating: OGD_Horizontals_Line.shp.xml  
  inflating: OGD_Horizontals_Line.shx  


In [5]:
# to write OGD_Horizontals.dbf to csv file ~ 2 minutes
filename= 'OGD_Horizontals.dbf'
table = DBF(filename)
f=open('ndWells.csv', 'w')
writer = csv.writer(f)
writer.writerow(table.field_names)
for record in table:
  writer.writerow(list(record.values()))

In [6]:
dfWells = pd.read_csv('ndWells.csv')
dfWells.head(3)

,wl_permit,api_wellno,api_format,long,lat,well_sub,measdpth,inclinatio,azimuth,tvd,coordns,coordnsdir,coordew,coordewdir,surveytype
0,1741,33007000260000,33-007-00026-00-00,-103.392973,46.866931,LAT1,9000.0,0.77,4.33,8999.030273,38.310001,N,35.090000,W,STP
1,1741,33007000260000,33-007-00026-00-00,-103.392971,46.866931,LAT1,9015.0,3.30,88.00,9014.021484,38.425602,N,34.650803,W,SPT
2,1741,33007000260000,33-007-00026-00-00,-103.392969,46.866931,LAT1,9022.0,5.30,88.00,9021.000977,38.443920,N,34.126297,W,SPT


In [18]:
dfWells['myLateral'] = ['1stLat' if x=='LAT1' else '1stLat' if x=='STK1' else 'VERT' if x=='VERT' else 'othLat' for x in dfWells['well_sub']]

In [19]:
dfWells.tail(3)

,wl_permit,api_wellno,api_format,long,lat,well_sub,measdpth,inclinatio,azimuth,tvd,coordns,coordnsdir,coordew,coordewdir,surveytype,myLateral
4885720,90466,33105904660000,33-105-90466-00-00,-103.418518,48.289651,LAT1,7360.0,88.900002,292.000000,5790.180664,663.569214,N,1681.323120,W,SPT,1stLat
4885721,90466,33105904660000,33-105-90466-00-00,-103.418636,48.289682,LAT1,7391.0,88.800003,291.799988,5790.802734,675.129395,N,1710.080322,W,SPT,1stLat
4885722,90466,33105904660000,33-105-90466-00-00,-103.418759,48.289715,LAT1,7423.0,88.800003,291.799988,5791.473145,687.010437,N,1739.785400,W,SPT,1stLat


In [20]:
flWells = dfWells.groupby(['api_wellno','myLateral']).agg(latMin=('lat', 'min'),latMax=('lat', 'max'),
                                                        longMin=('long', 'min'),longMax=('long', 'max'),
                                                        tvdMin=('tvd', 'min'),tvdMax=('tvd', 'max')).reset_index()

In [9]:
def latft(latMin, latMax, longMin, longMax):

    R = 20925721

    lat1 = radians(latMin)
    lon1 = radians(longMin)
    lat2 = radians(latMax)
    lon2 = radians(longMax)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    #print("latL_km:", distance)
    return distance

In [21]:
flWells['totll'] = flWells.apply(lambda row: latft(row['latMin'],row['latMax'],row['longMin'],row['longMax']),  axis=1)

In [22]:
flWells[['api_wellno','myLateral','totll']].to_csv('flWells.csv', index=False)

In [23]:
flWells[flWells['api_wellno']==33105040690000]

,api_wellno,myLateral,latMin,latMax,longMin,longMax,tvdMin,tvdMax,totll
37354,33105040690000,1stLat,48.282229,48.282416,-102.980406,-102.979728,7251.502930,8439.927734,178.459657
37355,33105040690000,VERT,48.282314,48.282738,-102.983372,-102.979511,1768.457642,8985.621094,951.301084


# **3D wellView**

In [26]:
plWell = dfWells[['api_wellno','lat','long','tvd','well_sub']][dfWells['api_wellno'] == 33105040690000]

In [27]:
fig = px.scatter_3d(plWell, x='long', y='lat', z='tvd', color='well_sub')
fig.update_traces(marker_size = 3)
fig.update_scenes(yaxis_autorange="reversed")
fig.update_scenes(xaxis_autorange="reversed")
fig.update_scenes(zaxis_range=[16000, 0])
fig.update_layout({
    'plot_bgcolor': 'rgba(100, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
                      })